**Decision Variables:**

- Let $X_A,X_B, \cdots X_E$ denote whether to install a sensor at each intersection. (Binary)


**Objective:**

$$ \text{Min. } 100X_A+150X_B+180X_C+160X_D+130X_E $$


**Constraints:**

$$\begin{aligned}
\text{(Monitoring A-B)} && X_A+X_B & \ge 1 \\
\text{(Monitoring A-C)} && X_A+X_C & \ge 1 \\
\text{(Monitoring B-D)} && X_B+X_D & \ge 1 \\
\text{(Monitoring C-D)} && X_C+X_D & \ge 1 \\
\text{(Monitoring D-E)} && X_D+X_E & \ge 1 \\
\text{(Monitoring C-E)} && X_C+X_E & \ge 1 
\end{aligned}$$

In [3]:
# Input data
intersections=['A','B','C','D','E']
import pandas as pd
cost=pd.Series([100,150,180,160,130],index=intersections)
segments=[['A','B'],['A','C'],['B','D'],['C','D'],['D','E'],['C','E']]

In [4]:
# Code to help you loop through segments
for segment in segments:
    beg,end=segment
    print(f'{beg}-{end}')

A-B
A-C
B-D
C-D
D-E
C-E


In [7]:
from gurobipy import Model, GRB

In [37]:
mod = Model()
x = mod.addVars(intersections, name = 'X', vtype = GRB.BINARY)
mod.update()
x

{'A': <gurobi.Var X[A]>,
 'B': <gurobi.Var X[B]>,
 'C': <gurobi.Var X[C]>,
 'D': <gurobi.Var X[D]>,
 'E': <gurobi.Var X[E]>}

In [38]:
obj = sum(cost[i]*x[i] for i in intersections)
obj

<gurobi.LinExpr: 100.0 X[A] + 150.0 X[B] + 180.0 X[C] + 160.0 X[D] + 130.0 X[E]>

In [39]:
for k in range(len(segments)):
    print(sum(x[i] for i in segments[k]))

<gurobi.LinExpr: X[A] + X[B]>
<gurobi.LinExpr: X[A] + X[C]>
<gurobi.LinExpr: X[B] + X[D]>
<gurobi.LinExpr: X[C] + X[D]>
<gurobi.LinExpr: X[D] + X[E]>
<gurobi.LinExpr: X[C] + X[E]>


In [40]:
sum(x[i] for i in segments[0])

<gurobi.LinExpr: X[A] + X[B]>

In [41]:
mod.setObjective(sum(cost[i]*x[i] for i in intersections))
for k in range(len(segments)):
    mod.addConstr(sum(x[i] for i in segments[k])>=1)
mod.write('10-5-example.lp')
%cat '10-5-example.lp'

\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  100 X[A] + 150 X[B] + 180 X[C] + 160 X[D] + 130 X[E]
Subject To
 R0: X[A] + X[B] >= 1
 R1: X[A] + X[C] >= 1
 R2: X[B] + X[D] >= 1
 R3: X[C] + X[D] >= 1
 R4: X[D] + X[E] >= 1
 R5: X[C] + X[E] >= 1
Bounds
Binaries
 X[A] X[B] X[C] X[D] X[E]
End


In [42]:
mod.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 6 rows, 5 columns and 12 nonzeros
Model fingerprint: 0xb941ebd5
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 460.0000000
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 9 nonzeros
Variable types: 0 continuous, 5 integer (5 binary)

Root relaxation: objective 3.900000e+02, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     390.0000000  390.00000  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 390 460 

Optimal solution fou

In [45]:
for b in intersections:
    print(f'x[{b}]: {x[b].x}')

x[A]: 1.0
x[B]: -0.0
x[C]: -0.0
x[D]: 1.0
x[E]: 1.0
